# TF Eager Tutorial (Define by Run)

정리 및 요약 by Ryah Shin

[참고1: 구글 블로그](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html)

[참고2: 구글 깃허브](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/g3doc/guide.md)

[Eager 모드 설치방법(TF Nightly)](https://github.com/tensorflow/tensorflow#installation)

In [59]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

# Numerical Computation

In [3]:
# Multiply 2 * 2 matrix
# Multiply two 2x2 matrices
x = tf.matmul([[1, 2],
               [3, 4]],
              [[4, 5],
               [6, 7]])

# Add one to each element
# (tf.add supports broadcasting)
#broadcasting: 퍼트리다의 의미, 크기가 작은 행렬을 크기가 큰 행렬로 맞추어 주는 기능, 축소는 데이터 손실 때문에 불가
#아래와 같은 경우는 1이 [[1 1], [1 1]], shape(2,2)로 변환
#차원이 다를 경우, expand_dims() 함수에 맞게 넣어야 함. 
#설명 참조: http://excelsior-cjh.tistory.com/entry/Matrix-Broadcasting-%ED%96%89%EB%A0%AC%EC%9D%98-%EB%B8%8C%EB%A1%9C%EB%93%9C%EC%BA%90%EC%8A%A4%ED%8C%85

y = tf.add(x, 1)

# Create a random random 5x3 matrix
z = tf.random_uniform([5, 3])

print(x)
print(y)
print(z)

tf.Tensor(
[[16 19]
 [36 43]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[17 20]
 [37 44]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 0.03383172  0.72217357  0.39446187]
 [ 0.7585969   0.81870437  0.45011866]
 [ 0.21634579  0.90425539  0.120924  ]
 [ 0.52222943  0.38636231  0.99817371]
 [ 0.58947456  0.60888505  0.22570443]], shape=(5, 3), dtype=float32)


In [6]:
# Tensor object를 활용하여, tf.add, tf.subtract, tf.multiply로 활용도 가능하다.
x = (tf.ones([1], dtype=tf.float32) + 1) * 2 - 1
print(x)

# Numpy를 활용하여 값을 변환하기
import numpy as np

x = tf.add(1, 1)                     # tf.Tensor with a value of 2
y = tf.add(np.array(1), np.array(1)) # tf.Tensor with a value of 2
z = np.multiply(x, y)                # numpy.int64 with a value of 

#반대로 tf.constant 활용하여 numpy -> tf로 변화
np_x = np.array(2., dtype=np.float32)
x = tf.constant(np_x)

py_y = 3.
y = tf.constant(py_y)

z = x + y + 1
print(z)
print(z.numpy())

tf.Tensor([ 3.], shape=(1,), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
6.0


## Define and Print Tensorflow Variables

In [ ]:
x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)
print(x)

#Tensorflow의 변수는 tensor로 나타냄으로, read_value()를 통해 현재 값으로 접근이 가능함.
#Tensorflow의 함수는 자동으로 초기화
print(x.read_value())

#numpy를 통한 변환
print(x.read_value().numpy())

#Tensorflow변수의 값을 변경하기
x.assign(42)
print(x.assign)

x.assign_add(3)
print(x.read_value())

#텐서 변수를 자유자제로 활용해보기
print(x + 3)

print(x * [1, 2, 4]) #자동으로 broadcasting해줌

## Automatic Difference (Gradients)

 - tfe.gradients_function(f): 입력 f에 대해 arg 미분값을 돌려준다.
 - tfe.value_and_gradients_function(f): tfe.gradients_function(f)과 비슷하지만, 함수가 들어오면 이전 f값과 미분값에 대해 돌려줌

In [55]:
def square(x):
    return tf.multiply(x, x)
assert 9 == square(3.).numpy()

grad = tfe.gradients_function(square)
assert 6 == grad(3.)[0].numpy()

print(square(3.))
print(grad(3.)) #x^2 -> 2x -> 6

#2차 gradients_function
grad2 = tfe.value_and_gradients_function(lambda x: grad(x)[0])
#assert 2 == grad2(3.)[0].numpy()
print("2nd grad: {}".format(grad2(3.)))

#3차 grad.
grad3 = tfe.gradients_function(lambda x: grad2(x)[0])
#assert 0 == grad3(3.)[0].numpy()
print(grad3(3.))

#absolute value
def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=19858, shape=(), dtype=float32, numpy=6.0>]
2nd grad: (<tf.Tensor: id=19867, shape=(), dtype=float32, numpy=6.0>, [<tf.Tensor: id=19874, shape=(), dtype=float32, numpy=2.0>])
[<tf.Tensor: id=19899, shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: id=19906, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=19915, shape=(), dtype=float32, numpy=-1.0>]


In [58]:
#실제 linear regression을 통하여 활용해보자

def prediction(input, weight, bias):
    return input * weight + bias

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# A loss function: Mean-squared error
def loss(weight, bias):
    error = prediction(training_inputs, weight, bias) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Function that returns the the derivative of loss with respect to
# weight and bias
grad = tfe.gradients_function(loss)

# Train for 200 steps (starting from some random choice for W and B, on the same
# batch of data).
W = 5.
B = 10.
learning_rate = 0.01
print("Initial loss: %f" % loss(W, B).numpy())
for i in range(200):
    (dW, dB) = grad(W, B)
    W -= dW * learning_rate
    B -= dB * learning_rate
    if i % 20 == 0:
        print("Loss at step %d: %f" % (i, loss(W, B).numpy()))
print("Final loss: %f" % loss(W, B).numpy())
print("W, B = %f, %f" % (W.numpy(), B.numpy()))

Initial loss: 67.847176
Loss at step 0: 65.231941
Loss at step 20: 29.920586
Loss at step 40: 14.025480
Loss at step 60: 6.866376
Loss at step 80: 3.640095
Loss at step 100: 2.185327
Loss at step 120: 1.528977
Loss at step 140: 1.232679
Loss at step 160: 1.098843
Loss at step 180: 1.038356
Final loss: 1.011916
W, B = 3.084922, 2.152886


## Custom Grad

Custom Gradient 제작하기. 

주로 cross entropy나 log likelyhood에 쓰이는 예제로 log(1 + e^x) 제작

In [10]:
def log1pexp(x):
    return tf.log(1 + tf.exp(x))

grad_log1pexp = tfe.gradients_function(log1pexp)

print(grad_log1pexp(0.)) # [0.5]

print(grad_log1pexp(100.)) # x = 100, nan

[<tf.Tensor: id=115, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=126, shape=(), dtype=float32, numpy=nan>]


In [13]:
@tfe.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad
grad_log1pexp = tfe.gradients_function(log1pexp)

# Gradient at x = 0 works as before.
print(grad_log1pexp(0.))
# [0.5]
# And now gradient computation at x=100 works as well.
print(grad_log1pexp(100.))
# [1.0]

[<tf.Tensor: id=138, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=150, shape=(), dtype=float32, numpy=1.0>]


## Building and training models

 - eager에서는 특별히 수정해야 하지 않는 한, tf.layers와 같은 모듈을 사용을 권장함
 - Optimizer와 layer를 간단하게 정리

## Variable & Optimization

 - tfe.Variable: 변형가능한 Tensor값을 저장하는 객체로써, 학습이나 미분을 할때 값에 대한 access가 가능함. 모델의 파라메터들이 python변수에 저장 될 수 있다는 이야기임
 - tfe.gradients_function(f): 쉬운 미분을 지원하지만, 모든 파라메터들이 f와 연동이 되어있어야 하여, 학습시 큰 파라메터에 대한 대응이 힘듬
 - tfe.implicit_gradients: 비슷한 기능이지만 몇가지 특수 기능이 있음?

In [51]:
#실제 linear regression을 통하여 활용해보자

class Model(object):
    def __init__(self):
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')

    def predict(self, inputs):
        return inputs * self.W + self.B


# The loss function to be optimized
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# Define:
# 1. A model
# 2. Derivatives of a loss function with respect to model parameters
# 3. A strategy for updating the variables based on the derivatives
model = Model()
grad = tfe.implicit_gradients(loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

# The training loop
print("Initial loss: %f" %
      loss(model, training_inputs, training_outputs).numpy())
for i in range(201):
    optimizer.apply_gradients(grad(model, training_inputs, training_outputs))
    if i % 20 == 0:
        print("Loss at step %d: %f" %
              (i, loss(model, training_inputs, training_outputs).numpy()))
print("Final loss: %f" % loss(model, training_inputs, training_outputs).numpy())
print("W, B = %s, %s" % (model.W.numpy(), model.B.numpy()))

Initial loss: 68.630684
Loss at step 0: 65.957047
Loss at step 20: 29.986712
Loss at step 40: 13.927011
Loss at step 60: 6.756695
Loss at step 80: 3.555240
Loss at step 100: 2.125806
Loss at step 120: 1.487559
Loss at step 140: 1.202574
Loss at step 160: 1.075323
Loss at step 180: 1.018501
Loss at step 200: 0.993129
Final loss: 0.993129
W, B = 3.02484, 2.1537


# Building Models (개선필요)

MNIST 2 Layer모델을 간단하게 Class로 만드는 예제

tfe.Network: 기본적으로 layer의 Container역할을 하여, 다른 NW객체에 임비디드 되어 NW객체가 된다.

추가로, inspection, saving, & restoring에 도움을 준다.

In [4]:
class MNISTModel(tfe.Network):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(units=10))
        self.layer2 = self.track_layer(tf.layers.Dense(units=10))
    def call(self, input):
        """모델 실행"""
        result = self.layer1(input)
        result = self.layer2(result)
        return result
    
#placeholder나 session에 대한 기능이 없고, input을 pass되면 자동으로 세팅 됨

In [5]:
# 테스트 데이터셋 생성하기
model = MNISTModel()
batch = tf.zeros([1, 1, 784])
print(batch.shape)
result = model(batch)
print(result)

(1, 1, 784)
tf.Tensor([[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]], shape=(1, 1, 10), dtype=float32)


In [8]:
#학습을 위한 loss func, grad, 그리고 업데이트

#1. loss func
def loss_function(model, x, y):
    y_ = model(x)
    return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)

#2. training loop
#implicit_gradients(): 모든 TF 값에 대한 미분을 계산한다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate =0.001)
for (x, y) in tfe.Iterator(dataset):
    grads = tfe.implicit_gradients(loss_function)(model, x, y)
    optimizer.apply_gradients(grads)

#GPU사용하기
#optimizer.min을 통해서, 짧게 작성하였지만, apply_gradients()기능을 써도 가능

with tf.device("/gpu:0"):
    for (x, y) in tfe.Iterator(dataset):
        optimizer.minimize(lambda: loss_function(model, x, y))

AttributeError: 'EagerTensor' object has no attribute '_as_variant_tensor'

## GPU 사용하기

 - eager에서는 GPU가 자동으로 실행되지 않기 때문에, 지정해주고 사용해야 한다.

In [10]:
#1000*1000 매트릭스 계산하기

import time

def measure(x):
    # The very first time a GPU is used by TensorFlow, it is initialized.
    # So exclude the first run from timing.
    tf.matmul(x, x)

    start = time.time()
    for i in range(10):
        tf.matmul(x, x)
    end = time.time()

    return "Took %s seconds to multiply a %s matrix by itself 10 times" % (end - start, x.shape)

# Run on CPU:
with tf.device("/cpu:0"):
    print("CPU: %s" % measure(tf.random_normal([1000, 1000])))

# If a GPU is available, run on GPU:
if tfe.num_gpus() > 0:
    with tf.device("/gpu:0"):
        print("GPU: %s" % measure(tf.random_normal([1000, 1000])))

CPU: Took 0.26638007164001465 seconds to multiply a (1000, 1000) matrix by itself 10 times


In [11]:
#Tensor 객체를 활용하여, 각 디바이스 별로 활용하는 것이 가능하다

x = tf.random_normal([10, 10])

x_gpu0 = x.gpu()
x_cpu = x.cpu()

_ = tf.matmul(x_cpu, x_cpu)  # Runs on CPU
_ = tf.matmul(x_gpu0, x_gpu0)  # Runs on GPU:0

if tfe.num_gpus() > 1:
    x_gpu1 = x.gpu(1)
    _ = tf.matmul(x_gpu1, x_gpu1)  # Runs on GPU:1

RuntimeError: Error copying tensor to device: GPU:0. GPU:0 unknown device.

## Using Eagar with Graphs

- Eagar 자체는 개발하고 디버깅 할 때 좋은 기능을 가지지만, Tensorflow graph형식이 분산 학습, 성능 최적화, 상용개발에 더 적합
- 현재 모델을 graph형태로 변경하기 위해서는, eager를 disable하고 실행하면 됨
- 관련 예제 코드: [MNIST with Eager](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/mnist)
- 위의 예제 코드는 checkpoints를 저장하고 불러올 수 있기 때문에, 상용에도 적합하다.

## 현재 활용하고 있는 코드의 변화
 
- 현재 사용하고 있는 데이터에서, 형태를 tf.data로 변경하는것을 추천 드립니다.
  - [참고링크 1](https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html)
  - [참고링크 2](https://www.tensorflow.org/programmers_guide/datasets)
- tf.layer.Conv2D()와 같은 Object-oriented 기능 활용을 추천 (Explict storage for variables
- 대부분의 모델이 eagar로 활용이 가능하지만, dynamic 모델에 대한 control flow같은 경우는 추가로 검토가 필요
- tfe.enable_eagar_execution()을 활용하면, 끌수가 없기 때문에 Python 세션을 재시작 추천

## 향후 To-Do

- 11/29 블로그를 지속적으로 백업하면서, 추가 개선 방향을 고민 할 것
 1. 데이터 전처리
 2. 제 3의 모델 (Conv Seq2Seq) 테스트
 3. 기존의 심심이 데이터에서 Generation 한 것이 의미가 있어 보임 -> 블로그도 Generation 후, 실제 의마기 있는 문장에 대한 데이터 추출로 보강